# JDE ETL Source Design
## Goal:  Generate source SQL with friendly names and built-in data Conversion
1. Pull *ALL* Field metadata based on QA 9.3:  Name, Datatype, Decimals
2. Pull *Specific* Table fields
3. Create SQL mapiing pull with data-conversion

In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import os, sys
import warnings

warnings.filterwarnings('ignore')

In [2]:
from decouple import config
config.search_path = '/home/jovyan/work'

### Connect to SQL DB

In [3]:
engine = create_engine(config('DATABASE_URL'))

### 1. Pull *ALL* Field metadata based on QA 9.3:  Name, Datatype, Decimals

In [4]:
sql_field_meta_server = 'ESYS_PROD'
sql_field_meta_lib = 'ARCPCOM71'
#sql_field_meta_lib = 'ARCPCOM71'
#sql_field_meta_lib = 'HSIPCOM93Q'


In [5]:
sql_field_meta = '''
SELECT 
	RTRIM("FRDTAI")				AS data_item
	,"FRDTAT"					AS data_item_type
	,"FROWTP"					AS data_type
	,"FRDTAS"					AS data_item_size
	,ISNULL("FRCDEC", 0)		AS display_decimals
	,ISNULL("FRDSCR", 'zNA')	AS row_description 
    
FROM 

    OPENQUERY ({}, '

	SELECT
		t.FRDTAI
		,FRDTAT
		,FROWTP
		,FRDTAS
		,FRCDEC
		,FRDSCR
	FROM
		{}.F9210 t
		LEFT JOIN {}.F9202 d
		ON t.FRDTAI = d.FRDTAI AND
			d.FRLNGP = '' '' AND
			d.FRSYR = '' ''  
')

'''.format(sql_field_meta_server, sql_field_meta_lib, sql_field_meta_lib)

In [6]:
print(sql_field_meta)


SELECT 
	RTRIM("FRDTAI")				AS data_item
	,"FRDTAT"					AS data_item_type
	,"FROWTP"					AS data_type
	,"FRDTAS"					AS data_item_size
	,ISNULL("FRCDEC", 0)		AS display_decimals
	,ISNULL("FRDSCR", 'zNA')	AS row_description 
    
FROM 

    OPENQUERY (ESYS_PROD, '

	SELECT
		t.FRDTAI
		,FRDTAT
		,FROWTP
		,FRDTAS
		,FRCDEC
		,FRDSCR
	FROM
		ARCPCOM71.F9210 t
		LEFT JOIN ARCPCOM71.F9202 d
		ON t.FRDTAI = d.FRDTAI AND
			d.FRLNGP = '' '' AND
			d.FRSYR = '' ''  
')




In [7]:
df_field_meta = pd.read_sql_query(sql_field_meta, engine);

In [8]:
df_field_meta.iloc[:,[2,3,4]] = df_field_meta.iloc[:,[2,3,4]].apply(lambda x: pd.to_numeric(x, errors='coerce'))
df_field_meta.fillna(value=0,inplace=True)

### 2. Pull *Specific* Table fields

#  Set Table Name HERE

In [9]:
sql_table = 'NPFIMS'

sql_link_server = 'ESYS_PROD'
sql_lib = 'ARCPTEL'
#sql_lib = 'ARCPDTA71'
#sql_lib = 'HSIPDTA71'


stage_db_schema = 'etl.'
convert_julian_date = True

# [ARC | HSI] [P | D] [DTA | CDC] [ 71 | 94]


In [10]:
sql_table_fields = '''

SELECT * from OPENQUERY ({}, '
	SELECT
		*
	FROM
		QSYS2.SYSCOLUMNS
	WHERE
        TABLE_SCHEMA = ''{}'' AND
		TABLE_NAME in( ''{}'')
    ORDER BY 
        ORDINAL_POSITION
')

''' .format(sql_link_server, sql_lib, sql_table)

In [11]:
#print (sql_table_fields)

In [12]:
df_table_fields = pd.read_sql_query(sql_table_fields, engine);

#### Join table fields with data dictionary meta-data

In [13]:
#df_table_fields

In [14]:
df_table_fields = df_table_fields[['ORDINAL_POSITION', 'COLUMN_NAME', 'COLUMN_TEXT', 'DATA_TYPE','LENGTH', 'NUMERIC_PRECISION']]

In [15]:
df_table_fields['data_item'] = df_table_fields.COLUMN_NAME.str[2:]

In [16]:
df_table_fields.head()

,ORDINAL_POSITION,COLUMN_NAME,COLUMN_TEXT,DATA_TYPE,LENGTH,NUMERIC_PRECISION,data_item
0,1,IMACDE,ACTIVITY CODE,CHAR,1,NaN,ACDE
1,2,IMACST,Item Average Cost,DECIMAL,9,9.0,ACST
2,3,IMACHF,AVAILABILITY CHANGE FLAG,CHAR,1,NaN,ACHF
3,4,IMAVLC,AVAILABILITY CODE CORPORATE,CHAR,1,NaN,AVLC
4,5,IMADTE,AVAILABILITY DATE,DECIMAL,6,6.0,ADTE


In [17]:
df_table_fields

,ORDINAL_POSITION,COLUMN_NAME,COLUMN_TEXT,DATA_TYPE,LENGTH,NUMERIC_PRECISION,data_item
0,1,IMACDE,ACTIVITY CODE,CHAR,1,NaN,ACDE
1,2,IMACST,Item Average Cost,DECIMAL,9,9.0,ACST
2,3,IMACHF,AVAILABILITY CHANGE FLAG,CHAR,1,NaN,ACHF
3,4,IMAVLC,AVAILABILITY CODE CORPORATE,CHAR,1,NaN,AVLC
4,5,IMADTE,AVAILABILITY DATE,DECIMAL,6,6.0,ADTE
5,6,IMRQTY,RESTRICTED QUANTITY,DECIMAL,5,5.0,RQTY
6,7,IMBODT,BACK-ORDER DATE,DECIMAL,6,6.0,BODT
7,8,IMCLSC,CLASS CODE,CHAR,2,NaN,CLSC
8,9,IMCQTC,CASE QUANTITY CODE,CHAR,1,NaN,CQTC
9,10,IMCLQT,CASE LOT QUANTITY,DECIMAL,4,4.0,CLQT


In [18]:
df_table_fields.data_item.unique()

array(['ACDE', 'ACST', 'ACHF', 'AVLC', 'ADTE', 'RQTY', 'BODT', 'CLSC',
       'CQTC', 'CLQT', 'DESC', 'DKYA', 'DKYB', 'HEIG', 'CSET', 'ITEM',
       'RSET', 'ITAX', 'KITC', 'LENG', 'NDC#', 'PINF', 'PCL1', 'WGTC',
       'SIZE', 'STRN', 'SCDE', 'VEND', 'VITM', 'WEIG', 'WIDT', 'DTYP',
       'LOCT', 'CLMJ', 'CLSJ', 'CLMC', 'CLSM', 'BUYR', 'SUPL', 'RPKE',
       'TXCA', 'UDEF', 'PUCD', 'ENTD', 'FRTC', 'HAZC', 'ADMC', 'CSTP',
       'PKTP', 'INVU', 'SLEL', 'JNKM', 'STCK', 'SLSC', 'PRCD', 'MSC1',
       'MSC2', 'MSC3', 'WDSC', 'PDSC', 'DCLS', 'PRLF', 'WCHC', 'NHAZ',
       'LCTL', 'PSLF', 'JDSC', 'LNTY', 'SNOR', 'STAS', 'SLBT', 'NDC1',
       'NDC2', 'NDC3', 'MLNK', 'ELEN', 'EWID', 'EHEI', 'GIN#', 'EDES',
       'IDES', 'PSPD', 'JSTN', 'WTYP', 'SIDS', 'US01', 'US02', 'US03',
       'US04', 'US05', 'US06', 'US07', 'US08', 'US09', 'US10', 'US11',
       'US12', 'US13', 'US14', 'US15'], dtype=object)

In [19]:
dff = pd.merge(df_table_fields,df_field_meta,on='data_item', how='left')
#dff = pd.merge(df_table_fields,df_field_meta,on='data_item', how='inner')


In [20]:
# hack to set resonable defaults for missing data
dff['display_decimals'].fillna(value='0',inplace=True)
dff['data_type'].fillna(value='0',inplace=True)
dff['row_description'].fillna(value=dff['COLUMN_TEXT'], inplace=True)

#### Cleanup final field name
remove trailing dots, special characters, and converto to lower_case

In [21]:
dff['row_description_final'] = dff.row_description.str.rstrip('. ').str.replace(r'%','pct').str.replace(r'$','amt').str.replace(r'[^0-9|a-z|" "]','', case=False).str.replace('  ',' ').str.lower().str.replace(' ','_')

#### Override with Defaults

In [22]:
dff['row_description_final'][dff['data_item']=='LITM'] = 'item_number'
dff['row_description_final'][dff['data_item']=='AN8'] = 'billto'
dff['row_description_final'][dff['data_item']=='SHAN'] = 'shipto'
dff['row_description_final'][dff['data_item']=='DOCO'] = 'salesorder_number'

### 3. Create SQL mapping pull with data-conversion

In [23]:
dff.groupby(['DATA_TYPE', 'data_type','display_decimals'])['ORDINAL_POSITION'].count()

DATA_TYPE  data_type  display_decimals
CHAR       1.0        0.0                  7
           2.0        0.0                 10
           9.0        0.0                 16
                      4.0                  1
           11.0       0.0                  1
           0          0                   44
DECIMAL    2.0        0.0                  1
           9.0        0.0                  5
                      4.0                  1
           0          0                   14
Name: ORDINAL_POSITION, dtype: int64

In [24]:
pd.options.display.max_rows = 99

In [25]:
dff

,ORDINAL_POSITION,COLUMN_NAME,COLUMN_TEXT,DATA_TYPE,LENGTH,NUMERIC_PRECISION,data_item,data_item_type,data_type,data_item_size,display_decimals,row_description,row_description_final
0,1,IMACDE,ACTIVITY CODE,CHAR,1,NaN,ACDE,A,2,12.0,0,Accounting Code. . . . . . . . . . . . .,accounting_code
1,2,IMACST,Item Average Cost,DECIMAL,9,9.0,ACST,P,9,15.0,4,Average Unit Cost. . . . . . . . . . . .,average_unit_cost
2,3,IMACHF,AVAILABILITY CHANGE FLAG,CHAR,1,NaN,ACHF,NaN,0,NaN,0,AVAILABILITY CHANGE FLAG,availability_change_flag
3,4,IMAVLC,AVAILABILITY CODE CORPORATE,CHAR,1,NaN,AVLC,A,1,1.0,0,Availability Check . . . . . . . . . . .,availability_check
4,5,IMADTE,AVAILABILITY DATE,DECIMAL,6,6.0,ADTE,NaN,0,NaN,0,AVAILABILITY DATE,availability_date
5,6,IMRQTY,RESTRICTED QUANTITY,DECIMAL,5,5.0,RQTY,P,9,15.0,0,Reorder Quantity . . . . . . . . . . . .,reorder_quantity
6,7,IMBODT,BACK-ORDER DATE,DECIMAL,6,6.0,BODT,NaN,0,NaN,0,BACK-ORDER DATE,backorder_date
7,8,IMCLSC,CLASS CODE,CHAR,2,NaN,CLSC,NaN,0,NaN,0,CLASS CODE,class_code
8,9,IMCQTC,CASE QUANTITY CODE,CHAR,1,NaN,CQTC,NaN,0,NaN,0,CASE QUANTITY CODE,case_quantity_code
9,10,IMCLQT,CASE LOT QUANTITY,DECIMAL,4,4.0,CLQT,P,9,15.0,0,Closing Quantity . . . . . . . . . . . .,closing_quantity


In [26]:
def field_format_sel(col_name, col_type, col_dec):
    val =''

    if col_type == 9 :
        if col_dec > 0 :
            val = 'CAST(({})/{} AS DEC({},{})) AS {}'.format(col_name,10**col_dec,15,np.int(col_dec),col_name)
        else :
            val = col_name
    elif  col_type == 11 : 
        if convert_julian_date :
            val = 'CASE WHEN {} IS NOT NULL THEN DATE(DIGITS(DEC({}+ 1900000,7,0))) ELSE NULL END AS {}'.format(col_name, col_name, col_name)
        else :
            val = '{} as {}'.format(col_name,col_name)
    else : 
        val = col_name
    return val;

def field_format_map(col_name, col_descr, is_etl = False):
    val =''

    col_name_format = '{message:{fill}{align}{width}}'.format(message=col_name, fill='_', align='<', width=6)
    
    col_descr_format = col_descr
    if str.isdigit(col_descr_format[0]) :
        col_descr_format = "_" + col_descr_format
        
    if is_etl :
        val = '{}_{} AS {}'.format(col_name_format, col_descr_format, col_descr_format)
    else :
        val = '"{}" AS {}_{}'.format(col_name, col_name_format, col_descr_format)

    return val;



sql_field_map = ', '.join([field_format_map(x,y)  \
                           for x, y in zip(dff['COLUMN_NAME'], dff['row_description_final'])])

sql_field_etl = ', '.join([field_format_map(x,y,is_etl=True)  \
                           for x, y in zip(dff['COLUMN_NAME'], dff['row_description_final'])])
#
# ok
sql_field_sel = ', '.join([ field_format_sel(col_name, col_type, col_dec) \
                           for col_name, col_type, col_dec in zip(dff['COLUMN_NAME'], dff['data_type'], dff['display_decimals'])])


In [27]:
sql_field_sel

'IMACDE, CAST((IMACST)/10000.0 AS DEC(15,4)) AS IMACST, IMACHF, IMAVLC, IMADTE, IMRQTY, IMBODT, IMCLSC, IMCQTC, IMCLQT, IMDESC, IMDKYA, IMDKYB, IMHEIG, IMCSET, IMITEM, IMRSET, IMITAX, IMKITC, IMLENG, IMNDC#, CAST((IMPINF)/10000.0 AS DEC(15,4)) AS IMPINF, IMPCL1, IMWGTC, IMSIZE, IMSTRN, IMSCDE, IMVEND, IMVITM, IMWEIG, IMWIDT, IMDTYP, IMLOCT, IMCLMJ, IMCLSJ, IMCLMC, IMCLSM, IMBUYR, IMSUPL, IMRPKE, IMTXCA, IMUDEF, IMPUCD, IMENTD, IMFRTC, IMHAZC, IMADMC, IMCSTP, IMPKTP, IMINVU, IMSLEL, IMJNKM, IMSTCK, IMSLSC, IMPRCD, IMMSC1, IMMSC2, IMMSC3, IMWDSC, IMPDSC, CASE WHEN IMDCLS IS NOT NULL THEN DATE(DIGITS(DEC(IMDCLS+ 1900000,7,0))) ELSE NULL END AS IMDCLS, IMPRLF, IMWCHC, IMNHAZ, IMLCTL, IMPSLF, IMJDSC, IMLNTY, IMSNOR, IMSTAS, IMSLBT, IMNDC1, IMNDC2, IMNDC3, IMMLNK, IMELEN, IMEWID, IMEHEI, IMGIN#, IMEDES, IMIDES, IMPSPD, IMJSTN, IMWTYP, IMSIDS, IMUS01, IMUS02, IMUS03, IMUS04, IMUS05, IMUS06, IMUS07, IMUS08, IMUS09, IMUS10, IMUS11, IMUS12, IMUS13, IMUS14, IMUS15'

In [28]:
sql_table_map = '''

--------------------------------------------------------------------------------
-- DROP TABLE STAGE_JDE_{}_{}_<instert_friendly_name_here>
--------------------------------------------------------------------------------

SELECT 

    {} 
    {} 

-- INTO {}{}_{}_<instert_friendly_name_here>

FROM 
    OPENQUERY ({}, '

	SELECT
		{}

	FROM
		{}.{}
--    WHERE
--        <insert custom code here>
--    ORDER BY
--        <insert custom code here>
')

--------------------------------------------------------------------------------

-- SELECT {} FROM <...>

'''.format(sql_lib, sql_table, 'Top 5', sql_field_map, stage_db_schema, sql_lib, sql_table, sql_link_server, sql_field_sel, sql_lib, sql_table, sql_field_etl)



### Output Table
1. Use to create STAGE via link and extract via IBM DTF 
1. Stub out missing fields from 7.1 -> 9.4, where and Autonum ID
1. Note that some Julian _JDT conversion will need to be converted Post 

In [29]:
print(sql_table_map)



--------------------------------------------------------------------------------
-- DROP TABLE STAGE_JDE_ARCPTEL_NPFIMS_<instert_friendly_name_here>
--------------------------------------------------------------------------------

SELECT 

    Top 5 
    "IMACDE" AS IMACDE_accounting_code, "IMACST" AS IMACST_average_unit_cost, "IMACHF" AS IMACHF_availability_change_flag, "IMAVLC" AS IMAVLC_availability_check, "IMADTE" AS IMADTE_availability_date, "IMRQTY" AS IMRQTY_reorder_quantity, "IMBODT" AS IMBODT_backorder_date, "IMCLSC" AS IMCLSC_class_code, "IMCQTC" AS IMCQTC_case_quantity_code, "IMCLQT" AS IMCLQT_closing_quantity, "IMDESC" AS IMDESC_description, "IMDKYA" AS IMDKYA_description_key_a, "IMDKYB" AS IMDKYB_description_key_b, "IMHEIG" AS IMHEIG_height, "IMCSET" AS IMCSET_cash_settlement_yn, "IMITEM" AS IMITEM_item_number, "IMRSET" AS IMRSET_rule_set_name, "IMITAX" AS IMITAX_item_tax_code, "IMKITC" AS IMKITC_kit_component_lines, "IMLENG" AS IMLENG_length, "IMNDC#" AS IMNDC#_ndc_code

In [30]:
dff

,ORDINAL_POSITION,COLUMN_NAME,COLUMN_TEXT,DATA_TYPE,LENGTH,NUMERIC_PRECISION,data_item,data_item_type,data_type,data_item_size,display_decimals,row_description,row_description_final
0,1,IMACDE,ACTIVITY CODE,CHAR,1,NaN,ACDE,A,2,12.0,0,Accounting Code. . . . . . . . . . . . .,accounting_code
1,2,IMACST,Item Average Cost,DECIMAL,9,9.0,ACST,P,9,15.0,4,Average Unit Cost. . . . . . . . . . . .,average_unit_cost
2,3,IMACHF,AVAILABILITY CHANGE FLAG,CHAR,1,NaN,ACHF,NaN,0,NaN,0,AVAILABILITY CHANGE FLAG,availability_change_flag
3,4,IMAVLC,AVAILABILITY CODE CORPORATE,CHAR,1,NaN,AVLC,A,1,1.0,0,Availability Check . . . . . . . . . . .,availability_check
4,5,IMADTE,AVAILABILITY DATE,DECIMAL,6,6.0,ADTE,NaN,0,NaN,0,AVAILABILITY DATE,availability_date
5,6,IMRQTY,RESTRICTED QUANTITY,DECIMAL,5,5.0,RQTY,P,9,15.0,0,Reorder Quantity . . . . . . . . . . . .,reorder_quantity
6,7,IMBODT,BACK-ORDER DATE,DECIMAL,6,6.0,BODT,NaN,0,NaN,0,BACK-ORDER DATE,backorder_date
7,8,IMCLSC,CLASS CODE,CHAR,2,NaN,CLSC,NaN,0,NaN,0,CLASS CODE,class_code
8,9,IMCQTC,CASE QUANTITY CODE,CHAR,1,NaN,CQTC,NaN,0,NaN,0,CASE QUANTITY CODE,case_quantity_code
9,10,IMCLQT,CASE LOT QUANTITY,DECIMAL,4,4.0,CLQT,P,9,15.0,0,Closing Quantity . . . . . . . . . . . .,closing_quantity


In [31]:
%time df_table_map = pd.read_sql_query(sql_table_map, engine);

OperationalError: (pymssql.OperationalError) (7342, b'An unexpected NULL value was returned for column "[MSDASQL].IMPINF" from OLE DB provider "MSDASQL" for linked server "ESYS_PROD". This column cannot be NULL.DB-Lib error message 20018, severity 16:\nGeneral SQL Server error: Check messages from the SQL Server\n') [SQL: '\n\n--------------------------------------------------------------------------------\n-- DROP TABLE STAGE_JDE_ARCPTEL_NPFIMS_<instert_friendly_name_here>\n--------------------------------------------------------------------------------\n\nSELECT \n\n    Top 5 \n    "IMACDE" AS IMACDE_accounting_code, "IMACST" AS IMACST_average_unit_cost, "IMACHF" AS IMACHF_availability_change_flag, "IMAVLC" AS IMAVLC_availability_check, "IMADTE" AS IMADTE_availability_date, "IMRQTY" AS IMRQTY_reorder_quantity, "IMBODT" AS IMBODT_backorder_date, "IMCLSC" AS IMCLSC_class_code, "IMCQTC" AS IMCQTC_case_quantity_code, "IMCLQT" AS IMCLQT_closing_quantity, "IMDESC" AS IMDESC_description, "I

### Next steps...
Add SQL to SQL Tools data package 

In [32]:
df_table_map

NameError: name 'df_table_map' is not defined